# Goal

1. Keywords similarity as the first run filtering
2. Distance in knowledge graph as the second run filtering

In [5]:
import os
import re
import pandas as pd
import json
from src.utils import read_config
from langchain.document_loaders import PyPDFLoader
from src.FastTextRank4Word import FastTextRank4Word
from src.FastTextRank4Sentence import FastTextRank4Sentence

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

In [6]:
configs = read_config('.env/configs.json')
DATA_PATH = configs["DATA_PATH"]
DB_PATH = configs['VECTORDB_PATH']
files_list = [file for file in os.listdir(DATA_PATH) if os.path.isfile(os.path.join(DATA_PATH, file))]

def read_pdf(files_list, data_path=DATA_PATH):
    content_dict = {}

    for file_name in files_list:
        file_path = data_path + file_name
        loader = PyPDFLoader(file_path)
        content = loader.load()
        whole_pdf = ""
        for i in range(len(content)):
            whole_pdf += content[i].page_content

        whole_pdf = re.sub(r'[：。\n\s]', '', whole_pdf)
        content_dict[file_name] = whole_pdf

    return content_dict

In [7]:
pdf_contents = read_pdf(files_list)

In [8]:
cleaned_text_1 = pdf_contents[list(pdf_contents.keys())[0]]
file_name_1 = list(pdf_contents.keys())[0]
cleaned_text_1

'1糾正案文（公布版）壹、被糾正機關國防部貳、案由國防部為國家中山科學研究院（下稱中科院）之監督機關，未能積極監督該院落實進用高齡人員實施計畫辦理人事相關事務，中科院一再打破進用計畫訂定高齡人員不任主管職之原則，致排擠現職人員升遷，打擊工作士氣並嚴重斲傷中科院形象；國防部復未能於103年中科院改制為行政法人初期，即督促該院建立完備之利衝規範，致中科院103至109年期間，得標廠商與該院現職人員存在多起利益衝突之不當情形；又國防部未能善盡監督責任，督促中科院建立離職人員的旋轉門規定，致有損及中科院自身利益之疑慮等，均核有失當，爰依法提案糾正參、事實與理由本案緣於近年審計部查核報告指出中科院行政法人化後衍生人事進用公正性及採購作業違反利益迴避等缺失，又媒體報導中科院大量回聘退休高階主管，淪肥貓院疑慮等情，已調查完畢國防部為中科院之監督機關，未能善盡監督責任，應予糾正促其注意改善，糾正之事實與理由如下一、中科院為因應營運急速擴張與各項重大任務推展，須以熟練人力即時投入工作，同時藉經驗傳承深化人才培育等需要，訂定進用高齡人員實施計畫，以曾在該院服務期間已核定為從事及參與機敏專案之涉密工項，或具有關鍵、高工藝、外部勞動市場稀少、短期內無法取得之專業技術者等退休人員列為聘任對象，2其立意良善；惟中科院卻一再打破該實施計畫所特別訂定高齡人員以不任主管職的原則，排擠現職人員升遷，打擊工作士氣，相關高齡回任人員雖於媒體揭露後已分別卸任主管職務，但已嚴重斲傷中科院形象，且動輒增設非常態之高階職缺予以安置，亦難杜因人設事之譏評又，竟有個別高齡回任者之議薪結果，違反該實施計畫規定，逾越原薪資之三分之二之上限，核其所為，已有不當濫用人事制度彈性、獨厚特定人之嫌，斷難以契約自由或並未違反法律等語卸責國防部為中科院之監督機關，未能積極監督該院落實進用高齡人員實施計畫辦理人事相關事務，以維該院正面形象與激勵現職人員之工作士氣，核有怠失(一)中科院依據勞動基準法第54條「勞工非有下列情形之一，雇主不得強制其退休一、年滿65歲者……」、中高齡者及高齡者就業促進法第28條「65歲以上勞工，雇主得以定期勞動契約僱用之」該院「定期契約人員管理作業規定」及「聘請顧問作業規章」等相關規定，由該院院長於109年2月17日主持之「定期契約及延後退休人力調節研處作為研討會議」後，於同年3月11日訂定「國家中山科學研究

In [9]:
cleaned_text_2 = pdf_contents[list(pdf_contents.keys())[1]]
file_name_2 = list(pdf_contents.keys())[1]
cleaned_text_2

'1糾正案文(公布版)壹、被糾正機關國防部暨所屬空軍司令部貳、案由國防部軍售案涉及預算排擠效應及兵力整建計畫建案作業程序，爰以特別預算編列，採緊急應變項次非特定需求方式辦理軍購，惟臺海戰情瞬息萬變，倘軍售案迭因偶發性事由，而造成武器無法依原初規劃籌獲、管理目標，未能達成整體防衛構想之作戰任務，將嚴重危害人員及國家安全；辦理空軍鳳翔專案採購，相關權責機關於審核採購計畫時均未就其所列非特定需求項目提出審核意見並請其修正或說明，即核定付諸實施，衍生在無需求品項執行下仍須計提美方管理成本之不合理情事；辦理空軍愛國者系統後勤勤務採購，以尚有餘款為由，修訂發價書調整新增工項採購年度計畫性射訓器材，違反規定將餘款轉運用，並增列不明確採購品項及數量之「非特定需求」項目達○○○萬餘美元，且違反該項金額不得超過全案總價○○％之規定，亦與國軍採購作業規定有違；該部空軍司令部身為武器系統、後勤事務之規劃、協調、管制、督導及軍品管理之規劃、督導及執行機關，對於所屬透過軍購籌建先進國防武器系統，報價審查及執行過程，仍有未臻周妥事項，皆核有重大違失，爰依法提案糾正參、事實與理由本案經調查竣事，確有下列失當之處，茲將事實及2理由臚列如後一、國防部軍售案涉及預算排擠效應及兵力整建計畫建案作業程序，爰以特別預算編列，採緊急應變項次非特定需求方式辦理軍購，惟臺海戰情瞬息萬變，倘軍售案迭因偶發性事由，而造成武器無法依原初規劃籌獲、管理目標，未能達成整體防衛構想之作戰任務，則將嚴重危害人員及國家安全，政府相關部門允應正視，務求萬全(一)中華民國憲法第137條規定「中華民國之國防，以保衛國家安全，維護世界和平為目的」國防法第2條規定「中華民國之國防，以發揮整體國力，建立國防武力，……，達成保衛國家與人民安全及維護世界和平之目的」同法第5條規定「中華民國陸海空軍，應服膺憲法，……，克盡職責，以確保國家安全」同法第7條規定「中華民國之國防體制，其架構如下︰一、總統二、國家安全會議三、行政院四、國防部」同法第10條規定「行政院制定國防政策，統合整體國力，督導所屬各機關辦理國防有關事務」同法第11條規定「國防部主管全國國防事務；……」(二)110年國防報告書1關此內容摘錄1、緒言面對快速變動且複雜的臺海與區域安全情勢，國軍秉持「自己國家自己救」的決心，以及「勿恃敵之不來，恃吾有以待之」精神，時時刻刻進行建軍備戰工作，

In [10]:
def content_extraction(input, pattern):

    # Search using the pattern
    match = re.search(pattern, input, re.S)

    if match:
        result = match.group(1).strip()  # Use strip() to remove any leading/trailing whitespace
        return result
    else:
        print("No match found")
        pass

In [11]:
def pattern_extraction(input):

    pattern = r"壹、被糾正機關(.*)貳、案"
    target = content_extraction(input, pattern)
    pattern = r"貳、案(.*?)參、事實.*?理由"
    reason = content_extraction(input, pattern)
    pattern = r"參、事實.*?理由(.*)"
    fact = content_extraction(input, pattern)


    return {"target": target, "reason": reason, "fact": fact}

In [12]:
def keyword_extraction(input, kw_num=10, ks_num=1):

    kw_model = FastTextRank4Word(tol=0.0001, window=5)
    kw=[kw_model.summarize(input['reason'], kw_num)][0]

    # ks_model = FastTextRank4Sentence(use_w2v=False,tol=0.0001)
    # ks=ks_model.summarize(input['fact'], ks_num)

    return { 
        "target": f"{input['target']}",
        "reason": f"{input['reason']}",
        "fact" : f"{input['fact']}",
        "keywords": f"{str(kw)}"
        }

In [13]:
def content_organize(input):

    result = {}

    for i, (name, content) in enumerate(input.items()):
        try:
            split = pattern_extraction(content)
            temp = keyword_extraction(split)
        except:
            print(f"{name} is problematic.")
            pass
        result[name] = temp

    print(f"{i+1} contents have been organized.")
    return result

In [14]:
organized_result = content_organize(dict(list(pdf_contents.items())))

No match found
No match found
南投某私中案糾正案文(公布版)(上網公布)_20230512160815252.pdf is problematic.
242 contents have been organized.


In [15]:
df = pd.DataFrame(organized_result).transpose()
df = df.reset_index().rename(columns={'index':'document'})

In [17]:
df.head()

,document,target,reason,fact,keywords
0,(公布版)中科院人員回聘案-糾正案文-(112國正6)_20230927160316477.pdf,國防部,由國防部為國家中山科學研究院（下稱中科院）之監督機關，未能積極監督該院落實進用高齡人員實施計...,本案緣於近年審計部查核報告指出中科院行政法人化後衍生人事進用公正性及採購作業違反利益迴避等缺...,"['中科院', '未能', '計畫', '建立', '督促', '利益', '進用', '主..."
1,(公布版)糾正案文(111國正3)_20231124100827470.pdf,國防部暨所屬空軍司令部,由國防部軍售案涉及預算排擠效應及兵力整建計畫建案作業程序，爰以特別預算編列，採緊急應變項次非...,本案經調查竣事，確有下列失當之處，茲將事實及2理由臚列如後一、國防部軍售案涉及預算排擠效應及...,"['需求', '管理', '軍購', '計畫', '品項', '建案', '年度', '工項..."
2,(公布版)陸軍官校案-糾正案文(112國正5)_20230818155815896.pdf,陸軍軍官學校,由陸軍軍官學校於109年10月7日晚間發生學弟以為學長慶生為由，由2名學弟壓著學長進入浴室洗...,有關「網傳陸軍官校學生疑似霸凌影片，畫面中2名學弟壓著學長洗澡，過程中拍打屁股、熱水澆淋身體...,"['影片', '案件', '事件', '屁股', '予以', '作息', '全裸', '制止..."
3,(公布版）空軍女中校糾正案（111國正2）_20230417101815845.pdf,國防部空軍司令部,由國防部空軍司令部未徵得所屬幹部自由意願，違法指派其擔任與軍人身分無關之差勤，擔任退休將領眷...,國防部空軍司令部指派一位女性中校軍官到醫院擔任退休將領配偶的醫療看護，並核予公假等情乙案，案...,"['退休', '公假', '工作', '指派', '差假', '相符', '查與', '顯有..."
4,(定稿)112司正0002_維民案糾正案文-公布版112.08.21 (修正3處112091...,行政院,由威權統治時期國家機關對於海外歸國學人返國服務進行安全查核，依調查局所查復維民專案不予介聘之...,有關「據訴，法務部調查局曾以維民專案對渠進行偵防調查，復製作不實調查報告，影響渠聲譽及求職，...,"['介聘', '人民', '不予', '政治', '思想', '箝制', '權之', '工作..."


In [16]:
df.to_csv('temp_result/organized_result_demo.csv', index=False, encoding='utf-8')

In [ ]:
# with open('temp_result/organized_result_2.txt', 'w', encoding='utf-8') as text_file:
#     json.dump(organized_result, text_file, ensure_ascii=False, indent=4)

In [ ]:
vectorizer = CountVectorizer().fit_transform([result_1['metadata']['kw'], result_2['metadata']['kw']])
vectors = vectorizer.toarray()

cosine_sim = cosine_similarity(vectors)

print(f"Cosine Similarity: {cosine_sim[0][1]}")

Cosine Similarity: 0.3


In [ ]:
vectors

array([[0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0]], dtype=int64)

# Knowledge graph

### Gemini

In [ ]:
gemini_key = os.get()

In [ ]:
def gemini_result_clean(gemini_result, gemini_key):
    url = f'https://generativelanguage.googleapis.com/v1/models/gemini-pro:generateContent?key={gemini_key}'
    headers = {'Content-Type': 'application/json'}
    data = {
        "contents": [
            {
                "parts": [{"text": """將以下內容調整成符合JSON規範的格式，以及補上entity1或是entity2，不要顯示JSON字樣""" + str(gemini_result)}]
            }
        ]
    }
    response = requests.post(url, headers=headers, json=data)
    print(f"response status_code: {response.status_code}")
    result = response.json()
    output = result['candidates'][0]['content']['parts'][0]['text']
    output = re.sub(r'[```json\n]','',output)
    
    return output

In [ ]:
def gemini_generation(target, ks, gemini_key):    

    url = f'https://generativelanguage.googleapis.com/v1/models/gemini-pro:generateContent?key={gemini_key}'
    headers = {'Content-Type': 'application/json'}

    data = {
        "contents": [
            {
                "parts": [{"text": """三元組指的是：實體1--關係--實體2。
                        實體通常是特定的人、組織或物體，關係則是描述兩個實體之間的關係，例如 {"實體1":"立法委員","關係":制定,"實體2":"法律"}
                            將以下這些内容以三元組方式描述並以Json格式呈現，只列出5組。用繁體中文回答:""" + str(ks)}]
            }
        ]
    }
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # This will raise an HTTPError for bad responses (4xx and 5xx)
        
        result_dict = response.json()
        extracted_result = result_dict['candidates'][0]['content']['parts'][0]['text']
        
        if '未找到三元組' in extracted_result:
            return None
        else:
            result_str = re.sub(r'[```json\n]', '', extracted_result).replace("實體", "entity").replace("關係", "relationship").split("、")[0]
            return result_str

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - Status code: {response.status_code}")
    except Exception as err:
        print(f"Other error occurred: {err}")

    gemini_output = gemini_result_clean(result_str)
    
    return gemini_output


In [ ]:
def knowledge_graph_json_generaton_gemini(input):
    output = input.copy()
    for name, value in input.items():
        target = value['target']
        ks = value['metadata']['ks']
        gemini_result = gemini_generation(target, ks, gemini_key)

        gemini_result = list(json.loads(gemini_result))[0]

        output[name]['knowledge_graph'] = gemini_result

    return output

In [ ]:
def process_knowledge_graph(gemini_result):
    # Attempt to load the JSON data
    try:
        loaded_data = json.loads(gemini_result)
    except json.JSONDecodeError:
        return []

    if isinstance(loaded_data, list):
        return loaded_data
    elif isinstance(loaded_data, dict) and '三元組' in loaded_data:
        return loaded_data['三元組']
    else:
        # Unknown structure
        return []

def ensure_entity_keys(item):
    if "entity1" not in item:
        item["entity1"] = item.pop("entity", None)
    if "entity2" not in item:
        item["entity2"] = item.pop("entity", None)
    return item

def filter_and_format_knowledge_graph(gemini_result):
    knowledge_graph = process_knowledge_graph(gemini_result)

    filtered_data = [
        ensure_entity_keys(item) for item in knowledge_graph
        if all(value is not None for value in item.values())
    ]

    return filtered_data


def extract_and_organize_json(organized_result, gemini_key=gemini_key):
    output = organized_result.copy()
    for name, value in organized_result.items():
        target = value['target']
        ks = value['metadata']['ks']
        gemini_result = gemini_generation(target, ks, gemini_key)

        processed_graph = filter_and_format_knowledge_graph(gemini_result)

        output[name]['knowledge_graph'] = processed_graph

    return output

In [ ]:
# with open('organized_data_1.txt', 'w', encoding='utf-8') as text_file:
#     json.dump(organized_result, text_file, ensure_ascii=False, indent=4)
# with open('organized_data_1.txt', 'r', encoding='utf-8') as text_file:
#     data = json.load(text_file)

In [ ]:
organized_final_result = extract_and_organize_json(organized_result)

In [ ]:
organized_final_result['34829.pdf']['knowledge_graph']

[{'entity1': '立法委員', 'relationship': '函轉', 'entity2': '民眾陳訴'},
 {'entity1': '行政院原住民族委員會', 'relationship': '補助', 'entity2': '花蓮區漁會'},
 {'entity1': '花蓮區漁會',
  'relationship': '採購',
  'entity2': '「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機」'},
 {'entity1': '行政院原住民族委員會', 'relationship': '監督', 'entity2': '花蓮區漁會'},
 {'entity1': '花蓮縣政府', 'relationship': '監督', 'entity2': '花蓮區漁會'}]

In [ ]:
with open('organized_final_result.txt', 'w', encoding='utf-8') as text_file:
    json.dump(organized_final_result, text_file, ensure_ascii=False, indent=4)

In [ ]:
"""
[{'entity1': '立法委員', 'relationship': '制定', 'entity2': '法律'},
 {'entity1': '聯勤總部暨所屬營工署',
  'relationship': '規劃不周未併案發包於先',
  'entity2': '林子口營區後續增設工程'},
 {'entity1': '營工署',
  'relationship': '未訂定施工所核對和地區工程處工地查驗作業時程',
  'entity2': '海軍神鷹二號新建水電工程及忠貞營區戰技館新建水電工程'},
 {'entity1': '地區工程處',
  'relationship': '完成查驗後，營工署審核流程和簽證作業時間過長',
  'entity2': '海軍神鷹二號新建水電工程及忠貞營區戰技館新建水電工程'},
 {'entity1': '營工署',
  'relationship': '承辦人員因公務差勤影響計價時效',
  'entity2': '海軍神鷹二號新建水電工程及忠貞營區戰技館新建水電工程'}]
"""

"\n[{'entity1': '立法委員', 'relationship': '制定', 'entity2': '法律'},\n {'entity1': '聯勤總部暨所屬營工署',\n  'relationship': '規劃不周未併案發包於先',\n  'entity2': '林子口營區後續增設工程'},\n {'entity1': '營工署',\n  'relationship': '未訂定施工所核對和地區工程處工地查驗作業時程',\n  'entity2': '海軍神鷹二號新建水電工程及忠貞營區戰技館新建水電工程'},\n {'entity1': '地區工程處',\n  'relationship': '完成查驗後，營工署審核流程和簽證作業時間過長',\n  'entity2': '海軍神鷹二號新建水電工程及忠貞營區戰技館新建水電工程'},\n {'entity1': '營工署',\n  'relationship': '承辦人員因公務差勤影響計價時效',\n  'entity2': '海軍神鷹二號新建水電工程及忠貞營區戰技館新建水電工程'}]\n"

In [ ]:
Prompt = """
根據以下內容列出三元組，以JSON方式呈現，若文章中的提及的對象跟target是同一個的話則替會成target
target: 行政院衛生署
內容：{content}
"""

In [ ]:
[
    {"subject": "政府", "predicate": "委託", "object": "行政院衛生署"},
    {"subject": "行政院衛生署", "predicate": "審慎衡酌考量", "object": "研究計畫"},
    {"subject": "行政院衛生署", "predicate": "刊登於機關網業", "object": "研究計畫主題及研究重點"},
    {"subject": "中央主管機關", "predicate": "實地查核", "object": "各機關委託研究計畫管理情形"},
    {"subject": "行政院衛生署", "predicate": "連續補助", "object": "相同單位或人員"},
    {"subject": "行政院衛生署", "predicate": "核有", "object": "連續二年度補助相同單位八十案"},
    {"subject": "行政院衛生署", "predicate": "核有", "object": "連續三年度補助相同單位三十八案"},
    {"subject": "行政院衛生署", "predicate": "核有", "object": "連續四年度補助相同單位二十四案"},
    {"subject": "行政院衛生署", "predicate": "向本院說明", "object": "補助計畫經公開徵求"},
    {"subject": "行政院衛生署", "predicate": "執行不同階段的任務", "object": "連續年度補助"},
    {"subject": "行政院衛生署", "predicate": "提出", "object": "九十至九十一年度實地查核報告"},
    {"subject": "行政院衛生署", "predicate": "提出", "object": "三項單一計畫實地查核報告"},
    {"subject": "行政院衛生署", "predicate": "補助", "object": "國立台灣師範大學林薇教授"},
    {"subject": "行政院衛生署", "predicate": "補助", "object": "生命期營養單張及指導手冊發展計畫"},
    {"subject": "行政院衛生署", "predicate": "檢送", "object": "重新編修的宣導單張及小冊"},
    {"subject": "行政院衛生署", "predicate": "查證稽核", "object": "委辦及補、捐助計畫執行情形"},
    {"subject": "行政院衛生署", "predicate": "列支", "object": "國外旅費"},
    {"subject": "行政院衛生署", "predicate": "規避", "object": "公教人員申請出國案件審核要點"}
]


[{'subject': '政府', 'predicate': '委託', 'object': '行政院衛生署'},
 {'subject': '行政院衛生署', 'predicate': '審慎衡酌考量', 'object': '研究計畫'},
 {'subject': '行政院衛生署', 'predicate': '刊登於機關網業', 'object': '研究計畫主題及研究重點'},
 {'subject': '中央主管機關', 'predicate': '實地查核', 'object': '各機關委託研究計畫管理情形'},
 {'subject': '行政院衛生署', 'predicate': '連續補助', 'object': '相同單位或人員'},
 {'subject': '行政院衛生署', 'predicate': '核有', 'object': '連續二年度補助相同單位八十案'},
 {'subject': '行政院衛生署', 'predicate': '核有', 'object': '連續三年度補助相同單位三十八案'},
 {'subject': '行政院衛生署', 'predicate': '核有', 'object': '連續四年度補助相同單位二十四案'},
 {'subject': '行政院衛生署', 'predicate': '向本院說明', 'object': '補助計畫經公開徵求'},
 {'subject': '行政院衛生署', 'predicate': '執行不同階段的任務', 'object': '連續年度補助'},
 {'subject': '行政院衛生署', 'predicate': '提出', 'object': '九十至九十一年度實地查核報告'},
 {'subject': '行政院衛生署', 'predicate': '提出', 'object': '三項單一計畫實地查核報告'},
 {'subject': '行政院衛生署', 'predicate': '補助', 'object': '國立台灣師範大學林薇教授'},
 {'subject': '行政院衛生署', 'predicate': '補助', 'object': '生命期營養單張及指導手冊發展計畫'},
 {'subject': '行政院衛生署', 'predi